In [21]:
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd
import re
from sklearn.metrics import log_loss

In [16]:
art_df = pd.read_csv('./articles_temp.csv')
ratings= pd.read_csv('./ratings_temp.csv')

In [17]:
ratings_clean = ratings['article']
articles_clean = art_df['content']

ratings_clean = re.sub("\d+", "", ratings_clean)
articles_clean = re.sub("\d+", "", articles_clean)

TypeError: expected string or bytes-like object

In [13]:
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1, 2))
X = tfidf.fit_transform(ratings_clean)
y = ratings['negative']
lr = LogisticRegression(solver='lbfgs')
lr.fit(X,y)

NameError: name 'ratings_clean' is not defined

In [8]:
art_df.dropna(inplace=True)

In [9]:
X_test = tfidf.transform()
rating_pred = lr.predict(X_test)
rating_lp = lr.predict_log_proba(X_test)

In [10]:
coef = pd.DataFrame(lr.coef_).T
coef.columns=['coefs']
coef['features']=tfidf.get_feature_names()

In [11]:
coef.sort_values(by='coefs').head(50)

,coefs,features
2674,-0.410609,facebook
3316,-0.391025,home
1743,-0.372900,committee
5759,-0.371518,remarks
6921,-0.340470,texas
4796,-0.333751,nthe
4573,-0.331938,nan
5378,-0.323366,president mike
2219,-0.311255,democratic
7720,-0.307337,vice president


In [12]:
coef.sort_values(by='coefs').tail(50)

,coefs,features
6093,0.217702,said support
6784,0.217702,support starting
5829,0.218596,representatives
1889,0.218920,control
3437,0.220346,ilhan omar
3436,0.220346,ilhan
7035,0.221865,today
4863,0.221878,officials
3514,0.225507,including
3253,0.225538,held


In [20]:
lr.score(X,y)

0.9646643109540636

In [27]:
log_loss([1,0]*2909,rating_lp)

0.6931471805599453